# ELE6310 - Assignment 2 - Data-Flow and Design Space

#### Name:
#### Student ID:

In [1]:
#@title Mount your Google Drive
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/ELE6310E/A2" #@param {type:"string"}
!ln -Ts $folder /content/A2 2> /dev/null

# Add the assignment folder to Python path
if '/content/A2' not in sys.path:
    sys.path.insert(0, '/content/A2')

# Install requirements
!pip install -qr /content/A2/requirements.txt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61

<ipython-input-2-2a541a4e5047>:21: UserWarning: CUDA is not available.
  warnings.warn('CUDA is not available.')


## 1- Structured Pruning

* First, complete `model_to_spars` and `generate_resnet_layers` in `solution. py`.

In [ ]:
# !git clone https://github.com/RedaBensaidDS/ELE6310E.git


In [ ]:
# !cp -v ${TL_INSTALL_PREFIX}/bin/accelergy* ${TL_EXEC_SAVE_PATH}/

In [3]:
import solution
import torch
import numpy as np
import random
from matplotlib import pyplot as plt
import os

## load data and model

In [4]:
from common.utils import load_CIFAR10_dataset, evaluate, fit, model_size
from common.resnet import resnet32

In [5]:
Seed = 6310
torch.manual_seed(Seed)
np.random.seed(Seed)
random.seed(Seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seed)
    torch.cuda.manual_seed_all(Seed)

In [6]:
train_loader, test_loader, calibration_loader = load_CIFAR10_dataset(batch_size=256, calibration_batch_size=1024)
model = resnet32(pretrained=True, save_path='./save/')
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

train_loader = [(images.to(device), labels.to(device)) for images, labels in train_loader]
test_loader = [(images.to(device), labels.to(device)) for images, labels in test_loader]

# accuracy = evaluate(model, test_loader, device)
# print("test accuracy of fp model:", accuracy, "model size:", model_size(model))


100%|██████████| 170498071/170498071 [00:01<00:00, 103091716.27it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet32-ef93fc4d.pt" to ./save/cifar10_resnet32-ef93fc4d.pt
100%|██████████| 1.85M/1.85M [00:00<00:00, 126MB/s]


CifarResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

* Use a method of your choice to find the optimal energy consumption for ResNet-32 with a constraint on test accuracy above 85%.

Any reasonable attempt at exploring the design space will give you full marks. Better approaches/results will be considered for bonus points.

In [7]:
prune_ratio_dict = {
    'conv1': 0.3,
    'layer1.0.conv1': 0.3,
    'layer1.0.conv2': 0.3,
    'layer1.1.conv1': 0.3,
    'layer1.1.conv2': 0.3,
    'layer1.2.conv1': 0.3,
    'layer1.2.conv2': 0.3,
    'layer1.3.conv1': 0.3,
    'layer1.3.conv2': 0.3,
    'layer1.4.conv1': 0.3,
    'layer1.4.conv2': 0.3,

    'layer2.0.conv1': 0.3,
    'layer2.0.conv2': 0.3,
    'layer2.0.downsample.0': 0.3,
    'layer2.1.conv1': 0.3,
    'layer2.1.conv2': 0.3,
    'layer2.2.conv1': 0.3,
    'layer2.2.conv2': 0.3,
    'layer2.3.conv1': 0.3,
    'layer2.3.conv2': 0.3,
    'layer2.4.conv1': 0.3,
    'layer2.4.conv2': 0.3,

    'layer3.0.conv1': 0.3,
    'layer3.0.conv2': 0.3,
    'layer3.0.downsample.0': 0.3,
    'layer3.1.conv1': 0.3,
    'layer3.1.conv2': 0.3,
    'layer3.2.conv1': 0.3,
    'layer3.2.conv2': 0.3,
    'layer3.3.conv1': 0.3,
    'layer3.3.conv2': 0.3,
    'layer3.4.conv1': 0.3,
    'layer3.4.conv2': 0.3
}


In [9]:
mapping = dict(enumerate(prune_ratio_dict.keys()))

In [8]:
prune_ratio_dict = {k:random.random() for k,v in prune_ratio_dict.items()}

In [10]:

sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
accuracy = evaluate(sparsed_model, test_loader, device)
print("test accuracy after pruning:", accuracy, "model size:", model_size(sparsed_model))


In [11]:
import torch.nn as nn

In [12]:
def get_shape_dict(sparsed_model):

  mod_name = {}
  for name, m in sparsed_model.named_modules():
      if isinstance(m, nn.Conv2d):
          if hasattr(m, 'weight_mask'):
            num_zero_channels = int(torch.sum(m.weight == 0).item()/(torch.tensor(m.weight.shape[1:]).prod().item()))
            mod_name[name]=(m.out_channels - num_zero_channels, m.in_channels, m.kernel_size[0], m.kernel_size[0])
  return mod_name


## Train functions

In [ ]:
def train(
        model: nn.Module,
        dataflow,
        criterion: nn.Module,
        optimizer,
        scheduler,
        device: torch.device = torch.device("cuda")
) -> None:
    model.train()

    for inputs, targets in dataflow:
        # # Move the data from CPU to GPU
        # inputs = inputs.to(device)
        # targets = targets.to(device)

        # Reset the gradients (from the last iteration)
        optimizer.zero_grad()

        # Forward inference
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward propagation
        loss.backward()

        # Update optimizer and LR scheduler
        optimizer.step()
        if scheduler is not None:
            scheduler.step()

In [ ]:
all_training_epochs = {}

In [ ]:
from tqdm.auto import tqdm
# from common.utils import train
def fit(model: nn.Module, num_epochs: int, train_loader, test_loader,
        criterion: nn.Module, optimizer, scheduler, device: torch.device = torch.device("cuda")):
    test_accuracy = []
    train_accuracy = []
    for epoch_num in tqdm(range(1, num_epochs + 1), desc="fit", leave=False):
        train(model, train_loader, criterion, optimizer, scheduler, device)
        metric = evaluate(model, train_loader, device)
        train_accuracy.append(metric)
        metric = evaluate(model, test_loader, device)
        test_accuracy.append(metric)
        print(f"epoch {epoch_num}: train_accuracy={train_accuracy[-1]}, test_accuracy={test_accuracy[-1]}")
        if metric >=85:
          break

        if len(test_accuracy) >= 2 and (abs(test_accuracy[-1] - test_accuracy[-2]) <0.001):
          break



    return metric, train_accuracy, test_accuracy

In [ ]:
def train_model(sparsed_model, max_epochs = 3):
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(sparsed_model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
  scheduler = None
  metric, train_accuracy, test_accuracy = fit(sparsed_model, max_epochs, train_loader, test_loader, criterion, optimizer, scheduler, device)
  return metric, train_accuracy, test_accuracy

In [ ]:
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(sparsed_model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
# scheduler = None
# metric = fit(sparsed_model, 1, train_loader, test_loader, criterion, optimizer, scheduler, device)

After fine-tuning, save the model and generate the YAML files for each layers of the pruned network. Then you can use `run_Accelergy` to estimate the energy consumption of pruned network.

In [ ]:
# train_model(sparsed_model)

In [ ]:
# accuracy = evaluate(sparsed_model, test_loader, device)
# print("test accuracy after pruning:", accuracy, "model size:", model_size(sparsed_model))

In [13]:
!source A2/install_timeloop/install_timeloop.sh
!export PATH=$PATH:~/.local/bin
!cp ~/.local/bin/timeloop* /usr/bin/
!cp ~/ENV/bin/accelergy* ~/.local/bin/
!cp ~/ENV/bin/accelergy* /usr/bin/
!which -a timeloop-model
!which -a accelergy

Creating symlink to /content/gdrive/MyDrive/ELE6310E/A2/install_timeloop
'/root/install_tl' -> '/content/gdrive/MyDrive/ELE6310E/A2/install_timeloop'
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-dev is already the newest version (1.74.0.3ubuntu7).
libboost-dev set to manually installed.
libboost-iostreams-dev is already the newest version (1.74.0.3ubuntu7).
libboost-iostreams-dev set to manually installed.
libboost-serialization-dev is already the newest version (1.74.0.3ubuntu7).
libboost-serialization-dev set to manually installed.
git is already the newest version (1:2.34.1-1ubuntu1.10).
libncurses-dev is already the newest version (6.3-2ubuntu0.1).
libncurses-dev set to manually installed.
The following additional packages will be installed:
  libconfig++9v5 libconfig-dev libconfig-doc libconfig9 libyaml-cpp0.7
  python3-setuptools python3-wheel
Suggested packag

In [14]:
solution.generate_resnet_layers(sparsed_model, base_path='/content/A2/common/layer_prob_base.yaml',  path='/content/A2/Q3/prob')
mod_name = get_shape_dict(sparsed_model)

In [ ]:
import subprocess
def Run_Accelergy(path_to_eyeriss_files='Q3', n=2, mod_name=None,lookup=None):
    current_path = os.getcwd()
    path_to_eyeriss_files = os.path.join(current_path, path_to_eyeriss_files)
    name_layers = os.listdir(os.path.join(path_to_eyeriss_files, 'prob'))
    os.system(f"rm -rf {current_path}/timeloop-model.stats.txt")
    energy_total = 0
    energy_dict = {}
    # print(name_layers)
    L = name_layers
    if n:
      L = name_layers[:n]
    for l in L:
        layer_name = l[:-5]
        if lookup and mod_name and lookup.get(mod_name.get(layer_name, False), False):
          print("found")

          energy = lookup.get(mod_name.get(layer_name, False), False)
        else:
          command = f"timeloop-mapper {path_to_eyeriss_files}/prob/{l} {path_to_eyeriss_files}/arch/components/*.yaml  " \
                    f"{path_to_eyeriss_files}/arch/eyeriss_like.yaml {path_to_eyeriss_files}/constraints/*.yaml  {path_to_eyeriss_files}/mapper/mapper.yaml" + " >/dev/null 2>&1"
          result = subprocess.run(command, shell=True, capture_output=True, text=True)
          energy = solution.Extract_Stats(path=f"{current_path}/timeloop-mapper.stats.txt")[0]
          if not mod_name.get(layer_name, False):
            print("**************************************",layer_name)
          lookup[mod_name.get(layer_name, False)] = energy
        print(f'{l[:-5]},{mod_name.get(layer_name, False)} : {energy} uJ')
        energy_total += energy
        os.system(f"rm -rf {current_path}/timeloop-model.stats.txt")
        energy_dict[l] = energy
    print(f"Total energy consumption for ResNet-32: {energy_total} uJ")
    return energy_total, energy_dict

In [29]:
# solution.Run_Accelergy('/content/A2/Q3')

## load saved pickles

In [ ]:
import pickle

with open("/content/A2/lookup_energy", 'wb') as file:
  pickle.dump(lookup, file)
lookup

In [19]:
import pickle
with open("/content/A2/lookup_energy", 'rb') as file:
  energy_lookup = pickle.load(file)

In [ ]:
import pickle
with open("/content/A2/acc_lookup_", 'rb') as file:
  acc_lookup = pickle.load(file)
acc_lookup = {}

In [ ]:
import pickle
with open("/content/A2/global_data_", 'rb') as file:
  global_data = pickle.load(file)
# global_data = {}

In [ ]:
import copy
import random
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


## Set random seed

In [ ]:
if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed = 777
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [16]:
full_mod_name = {'conv1': (16, 3, 3, 3),
 'layer1.0.conv1': (16, 16, 3, 3),
 'layer1.0.conv2': (16, 16, 3, 3),
 'layer1.1.conv1': (16, 16, 3, 3),
 'layer1.1.conv2': (16, 16, 3, 3),
 'layer1.2.conv1': (16, 16, 3, 3),
 'layer1.2.conv2': (16, 16, 3, 3),
 'layer1.3.conv1': (16, 16, 3, 3),
 'layer1.3.conv2': (16, 16, 3, 3),
 'layer1.4.conv1': (16, 16, 3, 3),
 'layer1.4.conv2': (16, 16, 3, 3),
 'layer2.0.conv1': (32, 16, 3, 3),
 'layer2.0.conv2': (32, 32, 3, 3),
 'layer2.0.downsample.0': (32, 16, 1, 1),
 'layer2.1.conv1': (32, 32, 3, 3),
 'layer2.1.conv2': (32, 32, 3, 3),
 'layer2.2.conv1': (32, 32, 3, 3),
 'layer2.2.conv2': (32, 32, 3, 3),
 'layer2.3.conv1': (32, 32, 3, 3),
 'layer2.3.conv2': (32, 32, 3, 3),
 'layer2.4.conv1': (32, 32, 3, 3),
 'layer2.4.conv2': (32, 32, 3, 3),
 'layer3.0.conv1': (64, 32, 3, 3),
 'layer3.0.conv2': (64, 64, 3, 3),
 'layer3.0.downsample.0': (64, 32, 1, 1),
 'layer3.1.conv1': (64, 64, 3, 3),
 'layer3.1.conv2': (64, 64, 3, 3),
 'layer3.2.conv1': (64, 64, 3, 3),
 'layer3.2.conv2': (64, 64, 3, 3),
 'layer3.3.conv1': (64, 64, 3, 3),
 'layer3.3.conv2': (64, 64, 3, 3),
 'layer3.4.conv1': (64, 64, 3, 3),
 'layer3.4.conv2': (64, 64, 3, 3)}

## DDPG

In [ ]:
class ReplayBuffer:
    """A simple numpy replay buffer."""

    def __init__(self, obs_dim: int, act_dim: int, size: int, batch_size: int = 32):
        """Initializate."""
        self.obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.next_obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.acts_buf = np.zeros([size, act_dim], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros([size], dtype=np.float32)
        self.max_size, self.batch_size = size, batch_size
        self.ptr, self.size, = 0, 0

    def store(
        self,
        obs: np.ndarray,
        act: np.ndarray,
        rew: float,
        next_obs: np.ndarray,
        done: bool,
    ):
        """Store the transition in buffer."""

        self.obs_buf[self.ptr] = obs
        self.next_obs_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample_batch(self) -> Dict[str, np.ndarray]:
        """Randomly sample a batch of experiences from memory."""
        idxs = np.random.choice(self.size, size=self.batch_size, replace=False)
        return dict(obs=self.obs_buf[idxs],
                    next_obs=self.next_obs_buf[idxs],
                    acts=self.acts_buf[idxs],
                    rews=self.rews_buf[idxs],
                    done=self.done_buf[idxs])

    def __len__(self) -> int:
        return self.size

In [ ]:
class OUNoise:
    """Ornstein-Uhlenbeck process.
    Taken from Udacity deep-reinforcement-learning github repository:
    https://github.com/udacity/deep-reinforcement-learning/blob/master/
    ddpg-pendulum/ddpg_agent.py
    """

    def __init__(
        self,
        size: int,
        mu: float = 0.0,
        theta: float = 0.15,
        sigma: float = 0.2,
    ):
        """Initialize parameters and noise process."""
        self.state = np.float64(0.0)
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self) -> np.ndarray:
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array(
            [random.random() for _ in range(len(x))]
        )
        self.state = x + dx
        return self.state

In [ ]:
class Actor(nn.Module):
    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        init_w: float = 3e-3,
    ):
        """Initialize."""
        super(Actor, self).__init__()

        self.hidden1 = nn.Linear(in_dim, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.out = nn.Linear(128, out_dim)

        self.out.weight.data.uniform_(-init_w, init_w)
        self.out.bias.data.uniform_(-init_w, init_w)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        # print(state, (state.squeeze()>=1).sum())
        # assert (state.squeeze()>=1).sum() == 0
        x = F.relu(self.hidden1(state))
        x = F.relu(self.hidden2(x))
        action = self.out(x).sigmoid()

        return action


class Critic(nn.Module):
    def __init__(
        self,
        in_dim: int,
        init_w: float = 3e-3,
    ):
        """Initialize."""
        super(Critic, self).__init__()

        self.hidden1 = nn.Linear(in_dim, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.out = nn.Linear(128, 1)

        self.out.weight.data.uniform_(-init_w, init_w)
        self.out.bias.data.uniform_(-init_w, init_w)

    def forward(
        self, state: torch.Tensor, action: torch.Tensor
    ) -> torch.Tensor:
        """Forward method implementation."""
        x = torch.cat((state, action), dim=-1)
        # assert (x.squeeze()>=1).sum() == 0
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        value = self.out(x)

        return value

In [ ]:
# acc_lookup={}
# global_data = {}

In [ ]:
class DDPGAgent:
    """DDPGAgent interacting with environment.

    Attribute:
        env (gym.Env): openAI Gym environment
        actor (nn.Module): target actor model to select actions
        actor_target (nn.Module): actor model to predict next actions
        actor_optimizer (Optimizer): optimizer for training actor
        critic (nn.Module): critic model to predict state values
        critic_target (nn.Module): target critic model to predict state values
        critic_optimizer (Optimizer): optimizer for training critic
        memory (ReplayBuffer): replay memory to store transitions
        batch_size (int): batch size for sampling
        gamma (float): discount factor
        tau (float): parameter for soft target update
        initial_random_steps (int): initial random action steps
        noise (OUNoise): noise generator for exploration
        device (torch.device): cpu / gpu
        transition (list): temporory storage for the recent transition
        total_step (int): total step numbers
        is_test (bool): flag to show the current mode (train / test)
    """
    def __init__(
        self,
        env,
        memory_size: int,
        batch_size: int,
        obs_dim: int,
        action_dim: int,
        ou_noise_theta: float,
        ou_noise_sigma: float,
        gamma: float = 0.99,
        tau: float = 5e-3,
        initial_random_steps: int = 1e4,
    ):
        """Initialize."""
        self.obs_dim = obs_dim
        self.action_dim = action_dim

        self.env = env
        self.memory = ReplayBuffer(obs_dim, action_dim, memory_size, batch_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.tau = tau
        self.initial_random_steps = initial_random_steps

        # noise
        self.noise = OUNoise(
            action_dim,
            theta=ou_noise_theta,
            sigma=ou_noise_sigma,
        )

        # device: cpu / gpu
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        print(self.device)

        # networks
        self.actor = Actor(obs_dim, action_dim).to(self.device)
        self.actor_target = Actor(obs_dim, action_dim).to(self.device)
        self.actor_target.load_state_dict(self.actor.state_dict())

        self.critic = Critic(obs_dim + action_dim).to(self.device)
        self.critic_target = Critic(obs_dim + action_dim).to(self.device)
        self.critic_target.load_state_dict(self.critic.state_dict())

        # optimizer
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=5e-4)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=5e-4)

        # transition to store in memory
        self.transition = list()

        # total steps count
        self.total_step = 0

        # mode: train / test
        self.is_test = False

    def select_action(self, state: np.ndarray) -> np.ndarray:
        """Select an action from the input state."""
        # if initial random action should be conducted
        if self.total_step < self.initial_random_steps and not self.is_test:
            selected_action = np.random.random(size=self.action_dim)
        else:

            selected_action = self.actor(
                torch.FloatTensor(state).to(self.device)
            ).detach().cpu().numpy()

        # add noise for exploration during training
        if not self.is_test:
            noise = self.noise.sample()
            selected_action = np.clip(selected_action + noise, 1/64, 63/64)

        self.transition = [state, selected_action]

        return selected_action

    def step(self, action: np.ndarray) -> Tuple[np.ndarray, np.float64, bool]:
        """Take an action and return the response of the env."""
        next_state, reward, done, _ = self.env.step(action)

        if not self.is_test:
            self.transition += [reward, next_state, done]
            self.memory.store(*self.transition)

        return next_state, reward, done

    def update_model(self) -> torch.Tensor:
        """Update the model by gradient descent."""
        device = self.device  # for shortening the following lines

        samples = self.memory.sample_batch()
        state = torch.FloatTensor(samples["obs"]).to(device)
        next_state = torch.FloatTensor(samples["next_obs"]).to(device)
        action = torch.FloatTensor(samples["acts"].reshape(-1, self.action_dim)).to(device)
        reward = torch.FloatTensor(samples["rews"].reshape(-1, 1)).to(device)
        done = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(device)

        masks = 1 - done
        next_action = self.actor_target(next_state)
        next_value = self.critic_target(next_state, next_action)
        curr_return = reward + self.gamma * next_value * masks

        # train critic
        values = self.critic(state, action)
        critic_loss = F.mse_loss(values, curr_return)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # train actor
        actor_loss = -self.critic(state, self.actor(state)).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # target update
        self._target_soft_update()

        return actor_loss.data, critic_loss.data

    def train(self, num_frames: int, max_steps: int, plotting_interval: int = 4):
        """Train the agent."""
        self.is_test = False
        state = self.env.reset()
        actor_losses = []
        critic_losses = []
        scores = []
        score = 0

        for self.total_step in range(1, num_frames + 1):

            if self.total_step % max_steps == 0 and self.total_step!=0:
              #episode end
              scores.append(score)
              score=0
              self._plot(
                    self.total_step,
                    scores,
                    actor_losses,
                    critic_losses,
                )
              state = self.env.reset()


              self.save_models("/content/A2/actor.pth", "/content/A2/critic.pth")
              with open("/content/A2/global_data_", 'wb') as file:
                pickle.dump(global_data, file)

              with open("/content/A2/acc_lookup_", 'wb') as file:
                pickle.dump(acc_lookup, file)

              with open("/content/A2/train_data_", 'wb') as file:
                pickle.dump(all_training_epochs, file)


              # with open("/content/A2/buffer_mem_", 'wb') as file:
              #   pickle.dump(self.memory, file)



            action = self.select_action(state)
            next_state, reward, done = self.step(action)





            # if training is ready
            if (
                len(self.memory) >= self.batch_size
                and self.total_step > self.initial_random_steps
            ):
                actor_loss, critic_loss = self.update_model()
                actor_losses.append(actor_loss)
                critic_losses.append(critic_loss)

            # plotting

            if self.total_step % plotting_interval == 0:
                print(f"********************{self.total_step} of {num_frames}**************************")
                print(next_state, -reward)

            state = next_state
            score += reward

    def _target_soft_update(self):
        """Soft-update: target = tau*local + (1-tau)*target."""
        tau = self.tau

        for t_param, l_param in zip(
            self.actor_target.parameters(), self.actor.parameters()
        ):
            t_param.data.copy_(tau * l_param.data + (1.0 - tau) * t_param.data)

        for t_param, l_param in zip(
            self.critic_target.parameters(), self.critic.parameters()
        ):
            t_param.data.copy_(tau * l_param.data + (1.0 - tau) * t_param.data)

    def _plot(
        self,
        frame_idx: int,
        scores: List[float],
        actor_losses: List[float],
        critic_losses: List[float],
    ):

        actor_losses = [i.to('cpu').numpy() for i in actor_losses]
        critic_losses = [i.to('cpu').numpy() for i in critic_losses]
        """Plot the training progresses."""
        def subplot(loc: int, title: str, values: List[float]):
            plt.subplot(loc)
            plt.title(title)
            plt.plot(values)

        subplot_params = [
            (131, f"frame {frame_idx}. score: {np.mean(scores[-10:])}", scores),
            (132, "actor_loss", actor_losses),
            (133, "critic_loss", critic_losses),
        ]

        plt.figure(figsize=(30, 5))
        for loc, title, values in subplot_params:
            subplot(loc, title, values)
        plt.show()


    def save_models(self, actor_filepath, critic_filepath):
        # Save actor and critic model state dictionaries
        torch.save(self.actor.state_dict(), actor_filepath)
        torch.save(self.critic.state_dict(), critic_filepath)
        print("Models saved successfully.")

    def load_models(self, actor_filepath, critic_filepath):
        # Load actor and critic model state dictionaries
        self.actor.load_state_dict(torch.load(actor_filepath, map_location=self.device))
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.critic.load_state_dict(torch.load(critic_filepath, map_location=self.device))
        self.critic_target.load_state_dict(self.critic.state_dict())
        print("Models loaded successfully.")

In [ ]:
counter = {}
for v in full_mod_name.values():
  counter[tuple(v[:])] = counter.get(tuple(v[:]), 0) + 1


state_max = 0
state_min = 0
for k1, v1 in counter.items():
  maxi = -float('inf')
  mini = float('inf')
  for k2, v2 in energy_lookup.items():
    if tuple(k2[1:]) == tuple(k1[1:]):
      if v2 >= maxi:
        maxi = v2
      if v2 <= mini:
        mini = v2
  state_max += maxi * v1
  state_min += mini * v1

print(state_max)
print(state_min)

2520.1700000000005
76.47999999999999


In [ ]:
# acc_lookup={}

In [ ]:
class Env:
  def __init__(self, observation_space, action_space, energy_lookup, acc_lookup):
    self.energy_lookup = energy_lookup
    self.acc_lookup = acc_lookup
    self.observation_space = np.zeros((observation_space,))
    self.action_space = np.zeros((action_space,))

  @staticmethod
  def step(action):
    action = action.squeeze()
    prune_ratio_dict = {mapping[i] : bit for i, bit in enumerate(action)}

    act_tuple = tuple([full_mod_name[mapping[i]][0] - int(np.round(full_mod_name[mapping[i]][0]*a)) for i, a in enumerate(action)])
    print(np.array(act_tuple))
    acc = acc_lookup.get(act_tuple, None)

    if acc is None:
      sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
      acc, train_accs, test_accs  = train_model(sparsed_model,5)
      all_training_epochs[act_tuple] = [train_accs, test_accs]
      acc_lookup[act_tuple] = acc


    mod_name = {k:(v[0] - int(np.round(prune_ratio_dict[k] * v[0])), *v[1:]) for k, v in full_mod_name.items()}
    energy = np.sum([energy_lookup.get(v, 10**7) for k, v in mod_name.items()])


    global_data[act_tuple] = (acc, energy)
    acc = int(acc)
    # reward = 1/(energy + 10000*max(85-acc, 0)) #1/(energy - state_min)/(state_max - state_min)
    next_state = np.array([[energy, acc]])
    mini = np.array([[state_min, 0]])
    maxi = np.array([[state_max, 100]])
    next_state = (next_state - mini)/(maxi-mini)
    reward = 1/(energy) + 1/(100*max(0.85-next_state.squeeze()[1], 0)+1)-1
    print("Energy: ",energy)
    return next_state, reward, False, None

  @staticmethod
  def reset():
    return np.array([[np.random.random(), np.random.random()]])

## Training

In [ ]:


env = Env(2,33,energy_lookup, None)


def seed_torch(seed):
    torch.manual_seed(seed)
    if torch.backends.cudnn.enabled:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

seed = 777
random.seed(seed)
np.random.seed(seed)
seed_torch(seed)

# parameters
num_frames = 1000
memory_size = 512
batch_size = 10
ou_noise_theta = 1.0
ou_noise_sigma = 0.1
initial_random_steps = 1

agent = DDPGAgent(
    Env,
    memory_size,
    batch_size,
    2,33,
    ou_noise_theta,
    ou_noise_sigma,
    initial_random_steps=initial_random_steps
)

cuda


In [ ]:
agent.load_models("/content/A2/actor.pth", "/content/A2/critic.pth")

Models loaded successfully.


In [ ]:
agent.train(num_frames, 5,1)

Output hidden; open in https://colab.research.google.com to view.